In [1]:
import ipywidgets as w

In [2]:
text_wg=w.Text(
    value="",
    placeholder="Path",
    description="Directory:",
    layout=w.Layout(width="400px")
)

In [3]:
import viser
import yourdfpy
import numpy as np
from viser.extras import ViserUrdf

In [4]:
import os
os.chdir('/home/hcis-s17/author_workdir/gymnasium_environments/')

In [5]:
load_meshes=True
load_collision_meshes=False
robot_urdf_object = yourdfpy.URDF.load(
    "/home/hcis-s17/author_workdir/gymnasium_environments/src/gymnasium_environments/meshes/franka_panda/robots/franka_panda_umi.urdf",
    load_meshes=load_meshes,
    build_scene_graph=load_meshes,
    load_collision_meshes=load_collision_meshes,
    build_collision_scene_graph=load_collision_meshes,
)


In [6]:
robot_urdf_object

In [7]:
def create_robot_control_sliders(
    server: viser.ViserServer, viser_urdf: ViserUrdf
) -> tuple[list[viser.GuiInputHandle[float]], list[float]]:
    slider_handles: list[viser.GuiInputHandle[float]] = []
    initial_config: list[float] = []
    for joint_name, (
        lower,
        upper,
    ) in viser_urdf.get_actuated_joint_limits().items():
        lower = lower if lower is not None else -np.pi
        upper = upper if upper is not None else np.pi
        initial_pos = 0.0 if lower < -0.1 and upper > 0.1 else (lower + upper) / 2.0
        slider = server.gui.add_slider(
            label=joint_name,
            min=lower,
            max=upper,
            step=1e-3,
            initial_value=initial_pos,
        )
        slider.on_update(  # When sliders move, we update the URDF configuration.
            lambda _: viser_urdf.update_cfg(
                np.array([slider.value for slider in slider_handles])
            )
        )
        slider_handles.append(slider)
        initial_config.append(initial_pos)
    return slider_handles, initial_config

In [8]:
server = viser.ViserServer(host="0.0.0.0", port=8000)
viser_urdf = ViserUrdf(
    server,
    urdf_or_path=robot_urdf_object,
    load_meshes=load_meshes,
    load_collision_meshes=load_collision_meshes,
    collision_mesh_color_override=(1.0, 0.0, 0.0, 0.5),
)

# Create sliders in GUI that help us move the robot joints.
with server.gui.add_folder("Joint position control"):
    (slider_handles, initial_config) = create_robot_control_sliders(
        server, viser_urdf
    )

# Add visibility checkboxes.
with server.gui.add_folder("Visibility"):
    show_meshes_cb = server.gui.add_checkbox(
        "Show meshes",
        viser_urdf.show_visual,
    )
    show_collision_meshes_cb = server.gui.add_checkbox(
        "Show collision meshes", viser_urdf.show_collision
    )

@show_meshes_cb.on_update
def _(_):
    viser_urdf.show_visual = show_meshes_cb.value

@show_collision_meshes_cb.on_update
def _(_):
    viser_urdf.show_collision = show_collision_meshes_cb.value

# Hide checkboxes if meshes are not loaded.
show_meshes_cb.visible = load_meshes
show_collision_meshes_cb.visible = load_collision_meshes

# Set initial robot configuration.
viser_urdf.update_cfg(np.array(initial_config))

# Create grid.
trimesh_scene = viser_urdf._urdf.scene or viser_urdf._urdf.collision_scene
server.scene.add_grid(
    "/grid",
    width=2,
    height=2,
    position=(
        0.0,
        0.0,
        # Get the minimum z value of the trimesh scene.
        trimesh_scene.bounds[0, 2] if trimesh_scene is not None else 0.0,
    ),
)

# Create joint reset button.
reset_button = server.gui.add_button("Reset")

@reset_button.on_click
def _(_):
    for s, init_q in zip(slider_handles, initial_config):
        s.value = init_q


try:
    server.scene.reset()
except NameError:
    server = viser.ViserServer()

╭──────────────── viser ────────────────╮
│             ╷                         │
│   HTTP      │ http://localhost:8000   │
│   Websocket │ ws://localhost:8000     │
│             ╵                         │
╰───────────────────────────────────────╯

In [11]:
from IPython.display import IFrame
app_url="http://localhost:8000"
IFrame(src=app_url, width='100%', height='700px')

In [9]:
def show_urdf(dirpath):
    pass